### Module 4 Project | Image Classification Algorithm | X-Ray Imaging to Detect Pneumonia Case

#### Goal

The goal is to create a machine learning algorithm that will successfully classify images into 2 categories.

* Positive for Pneumonia
* Negative for Pneumonia

The metric of success for this will be a high recall. This is due to the fact we want to reduce the number of False Negatives that are produced. We want to make sure we limit the amount of cases that are actually a pnemonia case, but labeled as healthy.

#### Methodology

The methodology that will be used for this project will be the OSEMN Methodology. I will outline the methodology below.

* Obtain: Gather data from relevant sources
* Scrub: Clean data to formats that machine understands
* Explore: Find significant patterns and trends using statistical methods
* Model: Construct models to predict and forecast
* iNterpret: Put results into good use

#### Data

This will be our Obtain step in our OSEMN as we did not need to do much data mining for this data set.

Our data was obtained from [Here](https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia) which is a dataset of thousands of chest x-ray images from both patients who have pnemonia and who do not. This labeled data will allow us to create a classification machine in order to determine whether a patient has a case of pnemonia based of a chest x-ray.

A larger set of the same data can be found [Here](https://data.mendeley.com/datasets/rscbjbr9sj/3). Due to technical limitations and the fact that there is no funds for this project we will not be using distributed networks to run this project, which limits the amount of data we can realistically work with while maintaining feasibility. 

In [1]:
#importing all neccessary libraries
from keras.preprocessing.image import ImageDataGenerator
import datetime
from keras import layers, models, optimizers
import matplotlib.pyplot as plt
%matplotlib inline


Using TensorFlow backend.


___
### Scrub

Here we will load the data, and begin to format it into the required format for our machine to understand it. 

First we need to load our images in order to begin processing our images down to the proper resolution, and begin creating our matrices for modeling.

We will also be timing our entire notebook to get a better sense of how long this would take beginning to finish.

In [3]:
# here we will instance 'current time' this will allow us to see how long the whole notebook takes
start_time = datetime.datetime.now()

Here we will begin to load in our data into 3 separate categories. 

* Training
* Validation
* Testing

In [4]:
#determine file paths
train_path = '../data/train'
test_path = '../data/test'
val_path = '../data/val'
#instance ImageDataGenerator for rescaling
train_IDG = ImageDataGenerator(rescale=1./255)
test_IDG = ImageDataGenerator(rescale=1./255)

# use our IDG to load and scale images

train_gen = train_IDG.flow_from_directory(train_path, target_size=(150,150), batch_size=20, class_mode='binary')

val_gen = test_IDG.flow_from_directory(val_path, target_size = (150,150), batch_size = 20, class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


Now that we have loaded our train and validation data we will begin to explore this data below. Once we have explored the data we will begin to create a baseline model, tune this model, then create a reusable pipeline to carryout or steps easily.

___
### Explore

This stage we will take a look into our data and see if we can notice anything. However the difficulty we will have here is that our data is in an image format, it could prove difficult to apply proper EDA to this data set. However, the main goal here is to express how it can prove difficult, and in some cases impossible to identify a case of pnemonia with the naked eye.

___
### Model

This is where we get into the meat and potatoes of our analysis. We will begin to test different algorithms in order to maximize the performance of our model. 

___
### iNterpret

Here we will begin to take a look ar our results. Understand what they mean, and how we can apply our outcome to larger datasets, and begin to look into productionizing our model.